In [1]:
import pandas as pd 
import numpy as np
import calendar
from datetime import datetime, date, timedelta

In [2]:
import ccxt
exchange_class = getattr(ccxt, 'binance')
binance  = exchange_class()


def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple())*1000
    ohlcv1 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv2 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv = ohlcv1 + ohlcv2
    return ohlcv

def ohlcv(dt, pair, period='1d'):
    ohlcv = []
    limit = 1000
    if period == '1m':
        limit = 720
    elif period == '1d':
        limit = 365
    elif period == '1h':
        limit = 24
    elif period == '5m':
        limit = 288
    for i in dt:
        start_dt = datetime.strptime(i, "%Y%m%d")
        since = calendar.timegm(start_dt.utctimetuple())*1000
        if period == '1m':
            ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        else:
            ohlcv.extend(binance.fetch_ohlcv(symbol=pair, timeframe=period, since=since, limit=limit))
    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)
    df.set_index('Time', inplace=True)
    return df

from datetime import datetime, date, timedelta

start_day = "20210510" #"20201201"
start_dt = datetime.strptime(start_day, "%Y%m%d")
end_day = "20220510"
end_dt = datetime.strptime(end_day, "%Y%m%d")
days_num = (end_dt - start_dt).days + 1
datelist = [start_dt + timedelta(days=x) for x in range(days_num)]
datelist = [date.strftime("%Y%m%d") for date in datelist]

#fwd = 10
coins = ['BTC' , 'ETH', 'FTM']
df_list = []
for coin in coins : 
  pair = coin + "/USDT"
  df = ohlcv(datelist, pair, '5m')

  for col in df.columns : 
    if col != "Time" : 
      df = df.rename(columns={col : col + "_" + coin})
    
  df_list.append(df)
    


In [3]:
df = df_list[0]
for i in range(len(coins) -1 ) : 
  df1 = df_list[i+1]
  df = pd.concat([df, df1], axis=1, sort=False)

  
df.to_csv("prices_all_5m.csv")